In [ ]:
!pip install ipywidgets plotly -q

import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import warnings
warnings.filterwarnings('ignore')

# Для интерактивных графиков
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Для виджетов
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Версии библиотек
print(f"TensorFlow: {tf.__version__}")
print(f"Keras: {keras.__version__}")

# Загружаем Fashion-MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Нормализация данных
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Добавляем размерность канала
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print(f"Размер обучающей выборки: {x_train.shape}")
print(f"Размер тестовой выборки: {x_test.shape}")

# Разделение на train/validation
val_split = 0.1
val_size = int(len(x_train) * val_split)
x_val = x_train[:val_size]
y_val = y_train[:val_size]
x_train = x_train[val_size:]
y_train = y_train[val_size:]

fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.ravel()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

for i in range(10):
    # Найдем первый пример каждого класса
    idx = np.where(y_train == i)[0][0]
    axes[i].imshow(x_train[idx].squeeze(), cmap='gray')
    axes[i].set_title(f'{class_names[i]}')
    axes[i].axis('off')

plt.suptitle('Примеры изображений Fashion-MNIST', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

stats_df = pd.DataFrame({
    'Класс': class_names,
    'Количество в обучающей': [np.sum(y_train == i) for i in range(10)],
    'Количество в тестовой': [np.sum(y_test == i) for i in range(10)]
})

print("Распределение по классам:")
print(stats_df)

input_shape = (28, 28, 1)
latent_dim = 32  # Размер латентного пространства

def build_basic_autoencoder(latent_dim=32):
    """Создание базового автокодировщика"""

    # Энкодер
    encoder_inputs = keras.Input(shape=input_shape)

    x = layers.Conv2D(32, 3, activation='relu', padding='same')(encoder_inputs)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2, padding='same')(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2, padding='same')(x)

    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.Flatten()(x)

    x = layers.Dense(256, activation='relu')(x)
    latent = layers.Dense(latent_dim, activation='relu', name='latent')(x)

    encoder = Model(encoder_inputs, latent, name='encoder')

    # Декодер
    latent_inputs = keras.Input(shape=(latent_dim,))

    x = layers.Dense(256, activation='relu')(latent_inputs)
    x = layers.Dense(7 * 7 * 128, activation='relu')(x)
    x = layers.Reshape((7, 7, 128))(x)

    x = layers.Conv2DTranspose(128, 3, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)

    x = layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(32, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)

    x = layers.Conv2DTranspose(32, 3, activation='relu', padding='same')(x)
    decoder_outputs = layers.Conv2D(1, 3, activation='sigmoid', padding='same')(x)

    decoder = Model(latent_inputs, decoder_outputs, name='decoder')

    # Автокодировщик
    autoencoder_outputs = decoder(encoder(encoder_inputs))
    autoencoder = Model(encoder_inputs, autoencoder_outputs, name='basic_autoencoder')

    return autoencoder, encoder, decoder

def build_sparse_autoencoder(latent_dim=32, sparsity_weight=0.001):
    """Создание разреженного автокодировщика"""

    # Энкодер с L1 регуляризацией
    encoder_inputs = keras.Input(shape=input_shape)

    x = layers.Conv2D(32, 3, activation='relu', padding='same')(encoder_inputs)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2, padding='same')(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2, padding='same')(x)

    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.Flatten()(x)

    x = layers.Dense(256, activation='relu')(x)

    # Добавляем L1 Регуляризацию:
    latent = layers.Dense(latent_dim, activation='relu',
                         activity_regularizer=keras.regularizers.l1(sparsity_weight),
                         name='latent')(x)

    encoder = Model(encoder_inputs, latent, name='sparse_encoder')

    # Декодер (такой же как в базовом)
    latent_inputs = keras.Input(shape=(latent_dim,))

    x = layers.Dense(256, activation='relu')(latent_inputs)
    x = layers.Dense(7 * 7 * 128, activation='relu')(x)
    x = layers.Reshape((7, 7, 128))(x)

    x = layers.Conv2DTranspose(128, 3, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)

    x = layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(32, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)

    x = layers.Conv2DTranspose(32, 3, activation='relu', padding='same')(x)
    decoder_outputs = layers.Conv2D(1, 3, activation='sigmoid', padding='same')(x)

    decoder = Model(latent_inputs, decoder_outputs, name='sparse_decoder')

    # Автокодировщик
    autoencoder_outputs = decoder(encoder(encoder_inputs))
    autoencoder = Model(encoder_inputs, autoencoder_outputs, name='sparse_autoencoder')

    return autoencoder, encoder, decoder

class Sampling(layers.Layer):
    """Слой для семплирования из латентного распределения"""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def build_vae(latent_dim=32):
    """Создание вариационного автокодировщика"""

    # Энкодер VAE
    encoder_inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(encoder_inputs)
    x = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)

    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
    z = Sampling()([z_mean, z_log_var])

    encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name='vae_encoder')

    # Декодер VAE
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(7 * 7 * 64, activation='relu')(latent_inputs)
    x = layers.Reshape((7, 7, 64))(x)
    x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
    decoder_outputs = layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)

    decoder = Model(latent_inputs, decoder_outputs, name='vae_decoder')

    # Создаем VAE модель
    vae_inputs = encoder_inputs
    vae_outputs = decoder(encoder(vae_inputs)[2])

    vae = Model(vae_inputs, vae_outputs, name='vae')

    return vae, encoder, decoder

def vae_loss(x, x_recon):
    """Функция потерь для VAE"""
    # Reconstruction loss
    reconstruction_loss = keras.losses.binary_crossentropy(x, x_recon)
    reconstruction_loss = tf.reduce_mean(reconstruction_loss) * 784

    # KL divergence loss
    return reconstruction_loss

# Дополнительная функция для вычисления KL потерь
def kl_loss(z_mean, z_log_var):
    """Вычисление KL divergence"""
    return -0.5 * tf.reduce_mean(
        1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
    )

print("Создание моделей...")
basic_ae, basic_encoder, basic_decoder = build_basic_autoencoder(latent_dim)
sparse_ae, sparse_encoder, sparse_decoder = build_sparse_autoencoder(latent_dim)
vae, vae_encoder, vae_decoder = build_vae(latent_dim)

print("\nАрхитектуры моделей:")
print(f"Базовый автокодировщик: {basic_ae.count_params()} параметров")
print(f"Разреженный автокодировщик: {sparse_ae.count_params()} параметров")
print(f"Вариационный автокодировщик: {vae.count_params()} параметров")

# Базовый автокодировщик
basic_ae.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Разреженный автокодировщик
sparse_ae.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Создаем VAE модель
class CustomVAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(CustomVAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        if isinstance(data, tuple):
            x, _ = data
        else:
            x = data

        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(x)
            reconstruction = self.decoder(z)

            reconstruction_loss = keras.losses.binary_crossentropy(x, reconstruction)
            reconstruction_loss = tf.reduce_mean(reconstruction_loss) * 784

            kl_loss = -0.5 * tf.reduce_mean(
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            )

            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {m.name: m.result() for m in self.metrics}

    def call(self, inputs):
        _, _, z = self.encoder(inputs)
        return self.decoder(z)


vae_custom = CustomVAE(vae_encoder, vae_decoder)
vae_custom.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))

batch_size = 128
epochs = 50

print("Обучение базового автокодировщика...")
basic_history = basic_ae.fit(
    x_train, x_train,
    validation_data=(x_val, x_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-6
        )
    ]
)

sparse_history = sparse_ae.fit(
    x_train, x_train,
    validation_data=(x_val, x_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-6
        )
    ]
)

vae.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy'
)

vae_history = vae.fit(
    x_train, x_train,
    validation_data=(x_val, x_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

fig, axes = plt.subplots(1, 3, figsize=(18, 4))

# Потери базового автокодировщика
axes[0].plot(basic_history.history['loss'], label='Обучающая', linewidth=2)
axes[0].plot(basic_history.history['val_loss'], label='Валидационная', linewidth=2)
axes[0].set_title('Базовый автокодировщик', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Эпоха')
axes[0].set_ylabel('Потери (MSE)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].fill_between(range(len(basic_history.history['loss'])),
                     basic_history.history['loss'],
                     basic_history.history['val_loss'],
                     alpha=0.2)

# Потери разреженного автокодировщика
axes[1].plot(sparse_history.history['loss'], label='Обучающая', linewidth=2)
axes[1].plot(sparse_history.history['val_loss'], label='Валидационная', linewidth=2)
axes[1].set_title('Разреженный автокодировщик', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Эпоха')
axes[1].set_ylabel('Потери (MSE)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].fill_between(range(len(sparse_history.history['loss'])),
                     sparse_history.history['loss'],
                     sparse_history.history['val_loss'],
                     alpha=0.2)

# Потери VAE
axes[2].plot(vae_history.history['loss'], label='Обучающая', linewidth=2)
axes[2].plot(vae_history.history['val_loss'], label='Валидационная', linewidth=2)
axes[2].set_title('Вариационный автокодировщик', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Эпоха')
axes[2].set_ylabel('Потери (Binary CE)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].fill_between(range(len(vae_history.history['loss'])),
                     vae_history.history['loss'],
                     vae_history.history['val_loss'],
                     alpha=0.2)

plt.suptitle('Кривые обучения разных архитектур автокодировщиков',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Примечание: VAE использует binary_crossentropy, другие - MSE
# Для сравнения нужно конвертировать потери

final_losses = pd.DataFrame({
    'Модель': ['Базовый', 'Разреженный', 'VAE'],
    'Обучающая потеря': [
        basic_history.history['loss'][-1],
        sparse_history.history['loss'][-1],
        vae_history.history['loss'][-1]
    ],
    'Валидационная потеря': [
        basic_history.history['val_loss'][-1],
        sparse_history.history['val_loss'][-1],
        vae_history.history['val_loss'][-1]
    ],
    'Тип потерь': ['MSE', 'MSE', 'Binary CE'],
    'Количество эпох': [
        len(basic_history.history['loss']),
        len(sparse_history.history['loss']),
        len(vae_history.history['loss'])
    ]
})

print("\nФинальные потери (разные метрики!):")
display(final_losses.style.format({
    'Обучающая потеря': '{:.6f}',
    'Валидационная потеря': '{:.6f}',
    'Количество эпох': '{:d}'
}).background_gradient(cmap='Reds_r', subset=['Обучающая потеря', 'Валидационная потеря']))

# Создаем директорию для сохранения моделей
os.makedirs('saved_models', exist_ok=True)

# Сохраняем веса моделей
basic_ae.save_weights('saved_models/basic_autoencoder.weights.h5')
sparse_ae.save_weights('saved_models/sparse_autoencoder.weights.h5')
vae.save_weights('saved_models/vae.weights.h5')

# Также сохраняем полные модели
basic_ae.save('saved_models/basic_autoencoder.keras')
sparse_ae.save('saved_models/sparse_autoencoder.keras')
vae.save('saved_models/vae.keras')

print("Модели сохранены в директории 'saved_models/':")
print("\nВ формате .weights.h5:")
print("1. basic_autoencoder.weights.h5")
print("2. sparse_autoencoder.weights.h5")
print("3. vae.weights.h5")
print("\nВ формате .keras:")
print("1. basic_autoencoder.keras")
print("2. sparse_autoencoder.keras")
print("3. vae.keras")

# Добавим вывод информации о классах
def compare_reconstructions_with_labels(models, model_names, n_images=5):
    """Сравнение реконструкций с выводом информации о классах"""

    # Выбираем случайные изображения разных классов
    indices = []
    selected_classes = []
    for class_id in range(10):
        class_indices = np.where(y_test == class_id)[0]
        if len(class_indices) > 0:
            idx = np.random.choice(class_indices)
            indices.append(idx)
            selected_classes.append(class_names[class_id])
        if len(indices) >= n_images:
            break

    # Если не нашли достаточно разных классов, добавляем случайные
    while len(indices) < n_images:
        idx = np.random.choice(len(x_test))
        indices.append(idx)
        selected_classes.append(class_names[y_test[idx]])

    indices = indices[:n_images]
    test_images = x_test[indices]

    print(f"\nВыбранные изображения и их классы:")
    for i, (idx, cls) in enumerate(zip(indices, selected_classes)):
        print(f"  {i+1}. Индекс {idx}: {cls}")

    fig, axes = plt.subplots(len(models) + 1, n_images, figsize=(15, 10))

    # Оригинальные изображения
    for i, (idx, cls) in enumerate(zip(indices, selected_classes)):
        axes[0, i].imshow(test_images[i].squeeze(), cmap='gray')
        axes[0, i].set_title(f'{cls}', fontsize=10)
        axes[0, i].axis('off')
        if i == n_images // 2:
            axes[0, i].set_title('Оригиналы', fontsize=12, fontweight='bold')

    # Реконструкции для каждой модели
    for model_idx, (model, name) in enumerate(zip(models, model_names), 1):
        reconstructions = model.predict(test_images, verbose=0)

        for i, (idx, cls) in enumerate(zip(indices, selected_classes)):
            axes[model_idx, i].imshow(reconstructions[i].squeeze(), cmap='gray')
            axes[model_idx, i].axis('off')

            # Рассчитываем MSE для этого изображения
            mse = np.mean((test_images[i] - reconstructions[i]) ** 2)
            psnr = 20 * np.log10(1.0 / np.sqrt(mse)) if mse > 0 else float('inf')
            axes[model_idx, i].set_title(f'MSE: {mse:.4f}', fontsize=9)

            if i == n_images // 2:
                axes[model_idx, i].set_ylabel(name, fontsize=11, fontweight='bold')

    plt.suptitle('Сравнение реконструкций разных автокодировщиков',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    # Анализ качества по классам
    print("\n" + "=" * 60)
    print("АНАЛИЗ КАЧЕСТВА РЕКОНСТРУКЦИИ ПО КЛАССАМ:")
    print("=" * 60)

    for model, name in zip(models, model_names):
        print(f"\n{name}:")
        reconstructions = model.predict(test_images, verbose=0)
        for i, (idx, cls) in enumerate(zip(indices, selected_classes)):
            mse = np.mean((test_images[i] - reconstructions[i]) ** 2)
            print(f"  • {cls}: MSE = {mse:.6f}")

# Запускаем с выводом классов
compare_reconstructions_with_labels(
    [basic_ae, sparse_ae, vae],
    ['Базовый', 'Разреженный', 'VAE'],
    n_images=5
)

def analyze_reconstruction_errors(n_samples=100):
    """Детальный анализ ошибок реконструкции"""

    # Выбираем случайные изображения
    indices = np.random.choice(len(x_test), n_samples, replace=False)
    test_images = x_test[indices]

    # Получаем реконструкции
    basic_recon = basic_ae.predict(test_images, verbose=0)
    sparse_recon = sparse_ae.predict(test_images, verbose=0)
    vae_recon = vae.predict(test_images, verbose=0)

    # Вычисляем ошибки
    basic_errors = np.abs(test_images - basic_recon)
    sparse_errors = np.abs(test_images - sparse_recon)
    vae_errors = np.abs(test_images - vae_recon)

    # Визуализация
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))

    # Распределение ошибок
    axes[0, 0].hist(basic_errors.flatten(), bins=50, alpha=0.7, label='Базовый', density=True)
    axes[0, 0].hist(sparse_errors.flatten(), bins=50, alpha=0.7, label='Разреженный', density=True)
    axes[0, 0].hist(vae_errors.flatten(), bins=50, alpha=0.7, label='VAE', density=True)
    axes[0, 0].set_xlabel('Абсолютная ошибка')
    axes[0, 0].set_ylabel('Плотность')
    axes[0, 0].set_title('Распределение ошибок реконструкции')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Кумулятивное распределение ошибок
    axes[0, 1].hist(basic_errors.flatten(), bins=50, alpha=0.7, label='Базовый',
                   cumulative=True, density=True, histtype='step', linewidth=2)
    axes[0, 1].hist(sparse_errors.flatten(), bins=50, alpha=0.7, label='Разреженный',
                   cumulative=True, density=True, histtype='step', linewidth=2)
    axes[0, 1].hist(vae_errors.flatten(), bins=50, alpha=0.7, label='VAE',
                   cumulative=True, density=True, histtype='step', linewidth=2)
    axes[0, 1].set_xlabel('Абсолютная ошибка')
    axes[0, 1].set_ylabel('Кумулятивная вероятность')
    axes[0, 1].set_title('Кумулятивное распределение ошибок')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Статистика ошибок по моделям
    error_stats = pd.DataFrame({
        'Модель': ['Базовый', 'Разреженный', 'VAE'],
        'Средняя ошибка': [
            np.mean(basic_errors),
            np.mean(sparse_errors),
            np.mean(vae_errors)
        ],
        'Максимальная ошибка': [
            np.max(basic_errors),
            np.max(sparse_errors),
            np.max(vae_errors)
        ],
        'Медианная ошибка': [
            np.median(basic_errors),
            np.median(sparse_errors),
            np.median(vae_errors)
        ],
        '95-й перцентиль': [
            np.percentile(basic_errors, 95),
            np.percentile(sparse_errors, 95),
            np.percentile(vae_errors, 95)
        ]
    })

    # Тепловая карта средних ошибок по классам
    class_errors = np.zeros((3, 10))  # 3 модели × 10 классов

    for class_id in range(10):
        class_indices = np.where(y_test[indices] == class_id)[0]
        if len(class_indices) > 0:
            class_imgs = test_images[class_indices]

            basic_class_recon = basic_ae.predict(class_imgs, verbose=0)
            sparse_class_recon = sparse_ae.predict(class_imgs, verbose=0)
            vae_class_recon = vae.predict(class_imgs, verbose=0)

            class_errors[0, class_id] = np.mean(np.abs(class_imgs - basic_class_recon))
            class_errors[1, class_id] = np.mean(np.abs(class_imgs - sparse_class_recon))
            class_errors[2, class_id] = np.mean(np.abs(class_imgs - vae_class_recon))

    im = axes[0, 2].imshow(class_errors, cmap='YlOrRd', aspect='auto')
    axes[0, 2].set_xticks(range(10))
    axes[0, 2].set_xticklabels([c[:3] for c in class_names], rotation=45)
    axes[0, 2].set_yticks(range(3))
    axes[0, 2].set_yticklabels(['Базовый', 'Разреженный', 'VAE'])
    axes[0, 2].set_title('Средняя ошибка по классам')
    plt.colorbar(im, ax=axes[0, 2])

    # Добавляем значения в ячейки
    for i in range(3):
        for j in range(10):
            axes[0, 2].text(j, i, f'{class_errors[i, j]:.3f}',
                           ha='center', va='center', color='black', fontsize=8)

    # Boxplot ошибок
    error_data = [basic_errors.flatten(), sparse_errors.flatten(), vae_errors.flatten()]
    axes[1, 0].boxplot(error_data, labels=['Базовый', 'Разреженный', 'VAE'])
    axes[1, 0].set_ylabel('Абсолютная ошибка')
    axes[1, 0].set_title('Boxplot ошибок реконструкции')
    axes[1, 0].grid(True, alpha=0.3, axis='y')

    # Scatter plot: MSE vs сложность изображения
    # Сложность = стандартное отклонение пикселей
    image_complexity = np.std(test_images, axis=(1, 2, 3))
    basic_mse_per_image = np.mean((test_images - basic_recon) ** 2, axis=(1, 2, 3))
    sparse_mse_per_image = np.mean((test_images - sparse_recon) ** 2, axis=(1, 2, 3))
    vae_mse_per_image = np.mean((test_images - vae_recon) ** 2, axis=(1, 2, 3))

    axes[1, 1].scatter(image_complexity, basic_mse_per_image, alpha=0.5, label='Базовый', s=20)
    axes[1, 1].scatter(image_complexity, sparse_mse_per_image, alpha=0.5, label='Разреженный', s=20)
    axes[1, 1].scatter(image_complexity, vae_mse_per_image, alpha=0.5, label='VAE', s=20)
    axes[1, 1].set_xlabel('Сложность изображения (std)')
    axes[1, 1].set_ylabel('MSE')
    axes[1, 1].set_title('Зависимость MSE от сложности изображения')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    # Пустая ось для статистики
    axes[1, 2].axis('off')
    stats_text = "Статистика ошибок:\n\n"
    for idx, row in error_stats.iterrows():
        stats_text += f"{row['Модель']}:\n"
        stats_text += f"  Среднее: {row['Средняя ошибка']:.4f}\n"
        stats_text += f"  Медиана: {row['Медианная ошибка']:.4f}\n"
        stats_text += f"  95%: {row['95-й перцентиль']:.4f}\n"
        stats_text += f"  Макс: {row['Максимальная ошибка']:.4f}\n\n"

    axes[1, 2].text(0.1, 0.5, stats_text, fontsize=10,
                   verticalalignment='center', fontfamily='monospace')

    plt.suptitle('Детальный анализ ошибок реконструкции',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    return error_stats

error_stats = analyze_reconstruction_errors(n_samples=200)

# @title **6.3 Визуализация латентного пространства**

def visualize_latent_space(encoder, x_data, y_data, title, is_vae=False, n_samples=500):
    """Визуализация латентного пространства"""

    # Используем подвыборку для ускорения
    x_subset = x_data[:n_samples]
    y_subset = y_data[:n_samples]

    print(f"Визуализация латентного пространства для {title}...")

    try:
        # Получаем латентные представления
        if is_vae:
            # Для VAE энкодер возвращает три значения
            predictions = encoder.predict(x_subset, verbose=0)
            if isinstance(predictions, list) and len(predictions) >= 3:
                z_mean = predictions[0]
                latent_vectors = z_mean
            else:
                latent_vectors = predictions
        else:
            latent_vectors = encoder.predict(x_subset, verbose=0)

        print(f"Размер латентных векторов: {latent_vectors.shape}")

        # Если размерность больше 2, берем первые 2 измерения
        if latent_vectors.shape[1] > 2:
            # Используем первые два латентных измерения
            latent_2d = latent_vectors[:, :2]
            dim_info = " (первые 2 измерения)"
        else:
            latent_2d = latent_vectors
            dim_info = ""

        fig, axes = plt.subplots(2, 3, figsize=(18, 10))

        # 1. Проекция на первые 2 латентных измерения
        scatter1 = axes[0, 0].scatter(latent_2d[:, 0], latent_2d[:, 1],
                                     c=y_subset, cmap='tab10', alpha=0.7, s=30)
        axes[0, 0].set_title(f'{title}\nПервые 2 латентных измерения{dim_info}', fontsize=12)
        axes[0, 0].set_xlabel('Латентное измерение 1')
        axes[0, 0].set_ylabel('Латентное измерение 2')
        axes[0, 0].grid(True, alpha=0.3)
        plt.colorbar(scatter1, ax=axes[0, 0], label='Класс')

        # 2. Случайная проекция 2D (берем случайные 2 измерения)
        if latent_vectors.shape[1] > 3:
            # Выбираем случайные 2 измерения
            import random
            dim1, dim2 = random.sample(range(latent_vectors.shape[1]), 2)
            scatter2 = axes[0, 1].scatter(latent_vectors[:, dim1], latent_vectors[:, dim2],
                                         c=y_subset, cmap='tab10', alpha=0.7, s=30)
            axes[0, 1].set_title(f'{title}\nСлучайные измерения {dim1} и {dim2}', fontsize=12)
            axes[0, 1].set_xlabel(f'Измерение {dim1}')
            axes[0, 1].set_ylabel(f'Измерение {dim2}')
            axes[0, 1].grid(True, alpha=0.3)
            plt.colorbar(scatter2, ax=axes[0, 1], label='Класс')
        else:
            axes[0, 1].axis('off')
            axes[0, 1].text(0.5, 0.5, 'Недостаточно измерений\nдля случайной проекции',
                           ha='center', va='center', fontsize=12)

        # 3. Гистограмма активаций латентных нейронов
        axes[0, 2].hist(latent_vectors.flatten(), bins=50, alpha=0.7, color='purple', density=True)
        axes[0, 2].set_xlabel('Значение активации')
        axes[0, 2].set_ylabel('Плотность')
        axes[0, 2].set_title('Распределение всех латентных активаций')
        axes[0, 2].grid(True, alpha=0.3)

        # Добавляем статистику на график
        mean_activation = np.mean(latent_vectors)
        std_activation = np.std(latent_vectors)
        axes[0, 2].axvline(mean_activation, color='red', linestyle='--',
                          label=f'Среднее: {mean_activation:.3f}')
        axes[0, 2].axvline(mean_activation + std_activation, color='orange', linestyle=':')
        axes[0, 2].axvline(mean_activation - std_activation, color='orange', linestyle=':')
        axes[0, 2].legend()

        # 4. Heatmap латентных активаций (первые 50 примеров, все измерения)
        n_examples = min(50, len(latent_vectors))
        n_dims = min(32, latent_vectors.shape[1])  # Показываем первые 32 измерения

        heatmap_data = latent_vectors[:n_examples, :n_dims]
        im = axes[1, 0].imshow(heatmap_data, aspect='auto', cmap='viridis',
                              interpolation='nearest')
        axes[1, 0].set_xlabel('Латентное измерение')
        axes[1, 0].set_ylabel('Пример')
        axes[1, 0].set_title(f'Heatmap латентных активаций\n(первые {n_examples} примеров, {n_dims} измерений)')
        plt.colorbar(im, ax=axes[1, 0])

        # 5. Средние активации по измерениям
        mean_per_dimension = np.mean(latent_vectors, axis=0)
        std_per_dimension = np.std(latent_vectors, axis=0)

        x_pos = np.arange(len(mean_per_dimension))
        axes[1, 1].bar(x_pos, mean_per_dimension, alpha=0.7, color='blue',
                      yerr=std_per_dimension, error_kw={'elinewidth': 1, 'capthick': 1})
        axes[1, 1].set_xlabel('Латентное измерение')
        axes[1, 1].set_ylabel('Средняя активация')
        axes[1, 1].set_title('Средние активации по измерениям')
        axes[1, 1].grid(True, alpha=0.3, axis='y')

        # 6. Статистика и выводы
        axes[1, 2].axis('off')

        # Вычисляем статистику
        threshold = 0.01
        sparsity = np.sum(np.abs(latent_vectors) < threshold) / latent_vectors.size * 100

        stats_text = f"СТАТИСТИКА ЛАТЕНТНОГО ПРОСТРАНСТВА:\n\n"
        stats_text += f"Размерность: {latent_vectors.shape[1]}\n"
        stats_text += f"Количество примеров: {len(latent_vectors)}\n"
        stats_text += f"Средняя активация: {mean_activation:.4f}\n"
        stats_text += f"Стандартное отклонение: {std_activation:.4f}\n"
        stats_text += f"Минимальная: {np.min(latent_vectors):.4f}\n"
        stats_text += f"Максимальная: {np.max(latent_vectors):.4f}\n"
        stats_text += f"Разреженность (<0.01): {sparsity:.1f}%\n\n"

        # Особенности по типу модели
        if "Разреженный" in title:
            stats_text += "ОЖИДАЕМЫЕ ОСОБЕННОСТИ:\n"
            stats_text += "Высокая разреженность\n"
            stats_text += "Много активаций ≈0\n"
            stats_text += "Немного сильных активаций\n"
        elif "VAE" in title:
            stats_text += "ОЖИДАЕМЫЕ ОСОБЕННОСТИ:\n"
            stats_text += "Нормальное распределение\n"
            stats_text += "Среднее ≈0\n"
            stats_text += "Хорошая структурированность\n"
        else:
            stats_text += "ОЖИДАЕМЫЕ ОСОБЕННОСТИ:\n"
            stats_text += "Равномерное распределение\n"
            stats_text += "Низкая разреженность\n"
            stats_text += "Меньше структуры\n"

        axes[1, 2].text(0.05, 0.95, stats_text, fontsize=10,
                       verticalalignment='top', fontfamily='monospace',
                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        plt.suptitle(f'Визуализация латентного пространства: {title}',
                     fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()

        # Выводим статистику в консоль
        print(f"Статистика латентного пространства для {title}:")
        print(f"Среднее значение активаций: {mean_activation:.4f}")
        print(f"Стандартное отклонение: {std_activation:.4f}")
        print(f"Минимальная активация: {np.min(latent_vectors):.4f}")
        print(f"Максимальная активация: {np.max(latent_vectors):.4f}")
        print(f"Разреженность (<0.01): {sparsity:.1f}%")

        if "Разреженный" in title:
            print(f"Процент нулевых активаций (<0.001): {np.sum(np.abs(latent_vectors) < 0.001) / latent_vectors.size * 100:.1f}%")
            print(f"Процент высоких активаций (>0.5): {np.sum(np.abs(latent_vectors) > 0.5) / latent_vectors.size * 100:.1f}%")

        return latent_vectors

    except Exception as e:
        print(f"Ошибка при визуализации для {title}: {e}")
        return None

print("=" * 60)
print("ВИЗУАЛИЗАЦИЯ ЛАТЕНТНЫХ ПРОСТРАНСТВ")
print("=" * 60)

n_samples = 300

# Визуализируем для каждой модели
try:
    print(f"1. Базовый автокодировщик (используем {n_samples} сэмплов):")
    basic_latent = visualize_latent_space(basic_encoder, x_test, y_test,
                                         'Базовый автокодировщик', n_samples=n_samples)

    print("\n" + "-" * 60)
    print(f"2. Разреженный автокодировщик (используем {n_samples} сэмплов):")
    sparse_latent = visualize_latent_space(sparse_encoder, x_test, y_test,
                                          'Разреженный автокодировщик', n_samples=n_samples)

    print("\n" + "-" * 60)
    print(f"3. Вариационный автокодировщик (VAE) (используем {n_samples} сэмплов):")
    vae_latent = visualize_latent_space(vae_encoder, x_test, y_test,
                                       'Вариационный автокодировщик', is_vae=True, n_samples=n_samples)

except NameError as e:
    print(f"Ошибка: {e}")
    print("Модели не определены. Вот что мы ожидаем увидеть:")

    print("\n" + "=" * 60)
    print("ТЕОРЕТИЧЕСКИЙ АНАЛИЗ ЛАТЕНТНЫХ ПРОСТРАНСТВ")
    print("=" * 60)

    print("1. БАЗОВЫЙ АВТОКОДИРОВЩИК:")
    print("   Распределение: равномерное или гауссовское")
    print("   Разреженность: низкая (10-30%)")
    print("   Структура: слабая кластеризация по классам")
    print("   Среднее: зависит от активации (ReLU → положительное)")

    print("\n2. РАЗРЕЖЕННЫЙ АВТОКОДИРОВЩИК:")
    print("   Распределение: много активаций ≈0, несколько сильных")
    print("   Разреженность: высокая (70-90%)")
    print("   Структура: более компактное представление")
    print("   Среднее: близко к 0 из-за регуляризации")

    print("\n3. VAE:")
    print("   Распределение: нормальное (гауссовское)")
    print("   Разреженность: средняя (20-50%)")
    print("   Структура: хорошо кластеризовано по классам")
    print("   Среднее: ≈0 (KL дивергенция к N(0,1))")

    basic_latent = sparse_latent = vae_latent = None

print("\n" + "=" * 60)
print("СРАВНЕНИЕ ЛАТЕНТНЫХ ПРОСТРАНСТВ")
print("=" * 60)

# Создаем сравнительную таблицу
if basic_latent is not None or sparse_latent is not None:
    print("СРАВНИТЕЛЬНЫЙ АНАЛИЗ ЛАТЕНТНЫХ ПРОСТРАНСТВ:")

    models_info = [
        ("Базовый", "Высокое качество реконструкции\nНизкая разреженность\nСлабая структурированность"),
        ("Разреженный", "Среднее/низкое качество\nВысокая разреженность\nКомпактные представления"),
        ("VAE", "Хорошее качество\nНормальное распределение\nХорошая кластеризация")
    ]

    for name, info in models_info:
        print(f"\n{name} автокодировщик:")
        print(info)

    print("\nВЫВОДЫ ДЛЯ ЗАДАЧИ СЖАТИЯ ДАННЫХ:")
    print("1. Базовый: лучший для точной реконструкции, но плохое сжатие")
    print("2. Разреженный: лучшее сжатие (много нулей), но потеря качества")
    print("3. VAE: баланс качества и структурированности")

print("\nВизуализация латентных пространств завершена")

def latent_interpolation_demo():
    """Демонстрация интерполяции в латентном пространстве"""

    print("=" * 60)
    print("ДЕМОНСТРАЦИЯ ИНТЕРПОЛЯЦИИ В ЛАТЕНТНОМ ПРОСТРАНСТВЕ")
    print("=" * 60)

    # Выбираем интересные пары классов для интерполяции
    interpolation_pairs = [
        (0, 1, "Футболка → Брюки"),
        (2, 4, "Свитер → Пальто"),
        (5, 7, "Сандалии → Кроссовки"),
        (6, 3, "Рубашка → Платье"),
        (8, 9, "Сумка → Ботильоны")
    ]

    for class1, class2, description in interpolation_pairs:
        print(f"\nИнтерполяция: {description}")

        # Находим примеры этих классов
        class1_indices = np.where(y_test == class1)[0]
        class2_indices = np.where(y_test == class2)[0]

        if len(class1_indices) > 0 and len(class2_indices) > 0:
            # Выбираем случайные примеры
            idx1 = np.random.choice(class1_indices)
            idx2 = np.random.choice(class2_indices)

            img1 = x_test[idx1:idx1+1]
            img2 = x_test[idx2:idx2+1]

            # Создаем фигуру: 3 строки (модели) × (n_steps + 2) столбцов
            n_steps = 8  # Уменьшим количество шагов для компактности
            fig, axes = plt.subplots(3, n_steps + 2, figsize=(n_steps + 4, 6))

            for model_idx, (encoder, decoder, model_name, is_vae) in enumerate([
                (basic_encoder, basic_decoder, 'Базовый', False),
                (sparse_encoder, sparse_decoder, 'Разреженный', False),
                (vae_encoder, vae_decoder, 'VAE', True)
            ]):
                # Получаем латентные представления
                if is_vae:
                    z1_mean, _, _ = encoder.predict(img1, verbose=0)
                    z2_mean, _, _ = encoder.predict(img2, verbose=0)
                    z1, z2 = z1_mean, z2_mean
                else:
                    z1 = encoder.predict(img1, verbose=0)
                    z2 = encoder.predict(img2, verbose=0)

                # Линейная интерполяция
                interpolated_imgs = []
                alphas = np.linspace(0, 1, n_steps)

                for alpha in alphas:
                    z_interp = (1 - alpha) * z1 + alpha * z2
                    recon = decoder.predict(z_interp, verbose=0)
                    interpolated_imgs.append(recon[0])

                # Визуализация - начало (индекс 0)
                axes[model_idx, 0].imshow(img1.squeeze(), cmap='gray')
                axes[model_idx, 0].set_title(f'{class_names[class1]}\nНачало', fontsize=8)
                axes[model_idx, 0].axis('off')

                # Интерполяции (индексы 1 до n_steps)
                for i in range(n_steps):
                    axes[model_idx, i + 1].imshow(interpolated_imgs[i].squeeze(), cmap='gray')
                    axes[model_idx, i + 1].axis('off')

                    # Подписываем середину интерполяции
                    if i == n_steps // 2 - 1:
                        axes[model_idx, i + 1].set_title(f'{model_name}\nα={alphas[i]:.1f}', fontsize=8)
                    else:
                        # Для остальных можно добавить значение alpha поменьше
                        if i < n_steps // 2:
                            axes[model_idx, i + 1].set_title(f'α={alphas[i]:.1f}', fontsize=7)

                # Конец (последний индекс)
                axes[model_idx, n_steps + 1].imshow(img2.squeeze(), cmap='gray')
                axes[model_idx, n_steps + 1].set_title(f'{class_names[class2]}\nКонец', fontsize=8)
                axes[model_idx, n_steps + 1].axis('off')

            plt.suptitle(f'Интерполяция: {description}', fontsize=14, fontweight='bold', y=1.02)
            plt.tight_layout()
            plt.show()

            # Анализ интерполяции
            print(f"  Класс {class1} → Класс {class2}")
            print(f"  Отображено для всех трех моделей")

            # Вычисляем и показываем MSE интерполяции
            print(f"  Статистика интерполяции:")
            for model_idx, (encoder, decoder, model_name, is_vae) in enumerate([
                (basic_encoder, basic_decoder, 'Базовый', False),
                (sparse_encoder, sparse_decoder, 'Разреженный', False),
                (vae_encoder, vae_decoder, 'VAE', True)
            ]):
                if is_vae:
                    z1_mean, _, _ = encoder.predict(img1, verbose=0)
                    z2_mean, _, _ = encoder.predict(img2, verbose=0)
                    z1, z2 = z1_mean, z2_mean
                else:
                    z1 = encoder.predict(img1, verbose=0)
                    z2 = encoder.predict(img2, verbose=0)

                # Евклидово расстояние между латентными векторами
                latent_distance = np.linalg.norm(z1 - z2)

                # Качество реконструкций начала и конца
                recon1 = decoder.predict(z1, verbose=0)
                recon2 = decoder.predict(z2, verbose=0)
                mse1 = np.mean((img1 - recon1) ** 2)
                mse2 = np.mean((img2 - recon2) ** 2)

                print(f"    {model_name}: расстояние={latent_distance:.2f}, MSE начала={mse1:.4f}, MSE конца={mse2:.4f}")
        else:
            print(f"  Не найдено примеров классов {class1} и {class2}")

# Запускаем демонстрацию интерполяции
latent_interpolation_demo()

def analyze_compression_efficiency():
    """Анализ эффективности сжатия"""

    # Параметры сжатия
    original_size = 28 * 28 * 8  # 28x28 пикселей × 8 бит на пиксель
    latent_size = latent_dim * 32  # 32 латентных признака × 32 бита на признак

    compression_ratio = original_size / latent_size
    space_saving = (1 - latent_size / original_size) * 100

    # Вычисляем качество для разных размеров батча
    batch_sizes = [1, 10, 100, 1000]

    results = []
    for batch_size_test in batch_sizes:
        test_subset = x_test[:batch_size_test]

        # Время предсказания
        import time

        # Базовый автокодировщик
        start = time.time()
        basic_recon = basic_ae.predict(test_subset, verbose=0)
        basic_time = time.time() - start

        # Разреженный автокодировщик
        start = time.time()
        sparse_recon = sparse_ae.predict(test_subset, verbose=0)
        sparse_time = time.time() - start

        # VAE
        start = time.time()
        vae_recon = vae.predict(test_subset, verbose=0)
        vae_time = time.time() - start

        # Качество
        basic_mse = np.mean((test_subset - basic_recon) ** 2)
        sparse_mse = np.mean((test_subset - sparse_recon) ** 2)
        vae_mse = np.mean((test_subset - vae_recon) ** 2)

        # Сохраняем результаты с правильными ключами
        results.append({
            'batch_size': batch_size_test,
            'basic_time': basic_time,
            'sparse_time': sparse_time,
            'vae_time': vae_time,
            'basic_mse': basic_mse,
            'sparse_mse': sparse_mse,
            'vae_mse': vae_mse
        })

    # Визуализация
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # 1. Время обработки vs размер батча
    batch_sizes = [r['batch_size'] for r in results]
    basic_times = [r['basic_time'] for r in results]
    sparse_times = [r['sparse_time'] for r in results]
    vae_times = [r['vae_time'] for r in results]

    axes[0, 0].plot(batch_sizes, basic_times, 'o-', label='Базовый', linewidth=2)
    axes[0, 0].plot(batch_sizes, sparse_times, 's-', label='Разреженный', linewidth=2)
    axes[0, 0].plot(batch_sizes, vae_times, '^-', label='VAE', linewidth=2)
    axes[0, 0].set_xlabel('Размер батча')
    axes[0, 0].set_ylabel('Время обработки (сек)')
    axes[0, 0].set_title('Производительность при разных размерах батча')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].set_xscale('log')
    axes[0, 0].set_yscale('log')

    # 2. Эффективность сжатия
    compression_data = pd.DataFrame({
        'Модель': ['Базовый', 'Разреженный', 'VAE'],
        'Степень сжатия': [compression_ratio, compression_ratio, compression_ratio],
        'Экономия памяти (%)': [space_saving, space_saving, space_saving],
        'Среднее время на изображение (мс)': [
            results[0]['basic_time'] * 1000,
            results[0]['sparse_time'] * 1000,
            results[0]['vae_time'] * 1000
        ]
    })

    x = range(3)
    width = 0.25

    axes[0, 1].bar([i - width for i in x], compression_data['Степень сжатия'],
                  width=width, label='Степень сжатия', alpha=0.7)
    axes[0, 1].bar(x, compression_data['Экономия памяти (%)'],
                  width=width, label='Экономия памяти (%)', alpha=0.7)
    axes[0, 1].bar([i + width for i in x], compression_data['Среднее время на изображение (мс)'],
                  width=width, label='Время (мс)', alpha=0.7)

    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(compression_data['Модель'])
    axes[0, 1].set_ylabel('Значение')
    axes[0, 1].set_title('Эффективность сжатия и производительность')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3, axis='y')

    # 3. Trade-off: качество vs скорость
    basic_speed = 1 / results[0]['basic_time'] if results[0]['basic_time'] > 0 else 0
    sparse_speed = 1 / results[0]['sparse_time'] if results[0]['sparse_time'] > 0 else 0
    vae_speed = 1 / results[0]['vae_time'] if results[0]['vae_time'] > 0 else 0

    axes[1, 0].scatter([basic_speed], [results[0]['basic_mse']], s=200,
                      label='Базовый', alpha=0.7)
    axes[1, 0].scatter([sparse_speed], [results[0]['sparse_mse']], s=200,
                      label='Разреженный', alpha=0.7)
    axes[1, 0].scatter([vae_speed], [results[0]['vae_mse']], s=200,
                      label='VAE', alpha=0.7)

    axes[1, 0].set_xlabel('Скорость (изображений/сек)')
    axes[1, 0].set_ylabel('MSE')
    axes[1, 0].set_title('Компромисс: качество vs скорость')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Добавляем аннотации
    for model, speed, mse in zip(['Базовый', 'Разреженный', 'VAE'],
                                [basic_speed, sparse_speed, vae_speed],
                                [results[0]['basic_mse'], results[0]['sparse_mse'], results[0]['vae_mse']]):
        axes[1, 0].annotate(model, (speed, mse), xytext=(5, 5),
                          textcoords='offset points', fontsize=9)

    # 4. Сводная таблица
    axes[1, 1].axis('off')

    summary_text = "СВОДКА ЭФФЕКТИВНОСТИ СЖАТИЯ:\n\n"
    summary_text += f"Исходный размер: {original_size} бит/изображение\n"
    summary_text += f"Сжатый размер: {latent_size} бит/изображение\n"
    summary_text += f"Степень сжатия: {compression_ratio:.1f}:1\n"
    summary_text += f"Экономия памяти: {space_saving:.1f}%\n\n"

    summary_text += "СРАВНЕНИЕ МОДЕЛЕЙ:\n"
    for model_name in ['Базовый', 'Разреженный', 'VAE']:
        summary_text += f"\n{model_name}:\n"

        # Получаем ключ для модели
        if model_name == 'Базовый':
            time_key = 'basic_time'
            mse_key = 'basic_mse'
        elif model_name == 'Разреженный':
            time_key = 'sparse_time'
            mse_key = 'sparse_mse'
        else:  # VAE
            time_key = 'vae_time'
            mse_key = 'vae_mse'

        # Используем результаты для batch_size=100 (индекс 2)
        if len(results) > 2:
            mse_value = results[2][mse_key]
            time_value = results[0][time_key]  # Для одного изображения

            summary_text += f"  MSE: {mse_value:.6f}\n"
            summary_text += f"  Время (1 img): {time_value*1000:.1f} мс\n"
            if time_value > 0:
                summary_text += f"  Скорость: {1/time_value:.1f} img/сек\n"
            else:
                summary_text += f"  Скорость: 0 img/сек\n"
        else:
            summary_text += f"  Данные недоступны\n"

    axes[1, 1].text(0.1, 0.5, summary_text, fontsize=10,
                   verticalalignment='center', fontfamily='monospace')

    plt.suptitle('Анализ эффективности сжатия данных',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    print("Ключевые показатели эффективности сжатия:")
    print(f"Степень сжатия: {compression_ratio:.1f}:1")
    print(f"Экономия памяти: {space_saving:.1f}%")
    print(f"Базовый автокодировщик: {1/results[0]['basic_time']:.1f} изображений/сек")
    print(f"Разреженный автокодировщик: {1/results[0]['sparse_time']:.1f} изображений/сек")
    print(f"VAE: {1/results[0]['vae_time']:.1f} изображений/сек")

# Запускаем анализ
analyze_compression_efficiency()

def interactive_reconstruction(model_choice, image_idx):
    """Интерактивное сравнение оригинального и реконструированного изображения"""

    models_dict = {
        'Базовый автокодировщик': basic_ae,
        'Разреженный автокодировщик': sparse_ae,
        'Вариационный автокодировщик': vae
    }

    model = models_dict[model_choice]

    # Выбираем изображение
    test_image = x_test[image_idx:image_idx+1]
    reconstruction = model.predict(test_image, verbose=0)[0]

    # Вычисляем разницу
    difference = np.abs(test_image[0] - reconstruction)

    # Визуализация
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()

    # Оригинал
    axes[0].imshow(test_image[0].squeeze(), cmap='gray')
    axes[0].set_title(f'Оригинал\nКласс: {class_names[y_test[image_idx]]}',
                      fontsize=12, fontweight='bold')
    axes[0].axis('off')

    # Реконструкция
    axes[1].imshow(reconstruction.squeeze(), cmap='gray')

    # Вычисляем метрики
    mse = np.mean((test_image[0] - reconstruction) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse)) if mse > 0 else float('inf')

    axes[1].set_title(f'{model_choice}\nMSE: {mse:.6f}\nPSNR: {psnr:.2f} dB',
                      fontsize=12, fontweight='bold')
    axes[1].axis('off')

    # Разница (цветная)
    im_diff = axes[2].imshow(difference.squeeze(), cmap='hot', vmin=0, vmax=1)
    axes[2].set_title('Абсолютная разница', fontsize=12, fontweight='bold')
    axes[2].axis('off')
    plt.colorbar(im_diff, ax=axes[2], fraction=0.046, pad=0.04)

    # Гистограмма различий
    axes[3].hist(difference.flatten(), bins=50, alpha=0.7, color='blue',
                edgecolor='black')
    axes[3].set_title('Распределение различий', fontsize=12, fontweight='bold')
    axes[3].set_xlabel('Величина различия')
    axes[3].set_ylabel('Частота')
    axes[3].grid(True, alpha=0.3)
    axes[3].axvline(difference.mean(), color='red', linestyle='--',
                   linewidth=2, label=f'Среднее: {difference.mean():.4f}')
    axes[3].legend()

    # Сравнение пиксель за пикселем
    axes[4].scatter(test_image[0].flatten(), reconstruction.flatten(),
                   alpha=0.3, s=1)
    axes[4].plot([0, 1], [0, 1], 'r--', linewidth=2, label='Идеальная реконструкция')
    axes[4].set_title('Сравнение пикселей', fontsize=12, fontweight='bold')
    axes[4].set_xlabel('Оригинальные значения')
    axes[4].set_ylabel('Реконструированные значения')
    axes[4].legend()
    axes[4].grid(True, alpha=0.3)

    # Метрики качества
    metrics_text = f"""
    Метрики качества:
    MSE: {mse:.6f}
    PSNR: {psnr:.2f} dB
    Макс. разница: {difference.max():.4f}
    Сред. разница: {difference.mean():.4f}
    Стандартное отклонение: {difference.std():.4f}
    """

    axes[5].text(0.1, 0.5, metrics_text, fontsize=11,
                verticalalignment='center', fontfamily='monospace')
    axes[5].set_title('Статистика', fontsize=12, fontweight='bold')
    axes[5].axis('off')

    plt.suptitle(f'Детальный анализ реконструкции для изображения {image_idx}',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

print("\nИспользуйте виджеты ниже для настройки:")
print("1. Выберите модель из выпадающего списка")
print("2. Измените номер изображения с помощью слайдера")

# Создаем интерактивные виджеты
interact(
    interactive_reconstruction,
    model_choice=widgets.Dropdown(
        options=['Базовый автокодировщик', 'Разреженный автокодировщик', 'Вариационный автокодировщик'],
        value='Базовый автокодировщик',
        description='Модель:',
        style={'description_width': 'initial'}
    ),
    image_idx=widgets.IntSlider(
        min=0,
        max=min(1000, len(x_test)-1),  # Ограничиваем для производительности
        value=0,
        description='Номер изображения:',
        continuous_update=False,
        style={'description_width': 'initial'}
    )
)